## **GIS 강의 2주차**

---

### **1. Geopandas 패키지 설치**
* 1-1) pip 로 Geopandas 설치하기
* 1-2) import library
### **2. 파일 불러오기**
* 2-1) shp 파일 불러오기 
* 2-2) csv 파일 불러오기 

### **3. 공간 데이터 핸들링**
* 3-1) 데이터 핸들링 
* 3-2) csv 파일의 x,y 컬럼을 공간 데이터(GeoDataframe) 로 변환 
* 3-3) 좌표계 부여, 변환하기 

### **4. Geopandas로 공간 속성 다루기**
* 4-1) 면적 (area) 
* 4-2) 길이
* 4-3) 도형의 무게중심
* 4-4) boundary 확인 
* 4-5) 공간의 유효성 확인 
* 4-6) 유효한 도형으로 만들기 

### **5. shapley 패키지 사용하여 shape 만들기**
* 5-1) 점(Point)
* 5-2) 선(LineString) 
* 5-3) 도형(Polygon) 
* 5-4) 한번에 여러 도형 만들기 (점,선,면))

### **6. 공간데이터 활용하기**
* 6-1) 버퍼(buffer)
* 6-2) 도형을 감싸는 사각형 그리기(envelope) 
* 6-3) 도형 융해(dissolve)
* 6-4) 도형끼리 겹치는 부분 처리하기(overlay)

### **7. 공간 데이터 결합**
* 7-1) 여러 공간 결합 방법들 
* 7-2) 실제 데이터 응용하기

### **8. 저장하기**
* 8-1) shp 파일로 저장하기 

### **9. Reference**
* 9-1) ref 

차주 목표 : folium 패키지를 이용한 지도 시각화, plotly 를 통한 interactive 한 시각화, 공간통계 분석방법 

----


### **1. Geopandas 패키지 설치**
#### 1-1) pip 로 Geopandas 설치하기 <br>
  pip install 로 geopandas 를 설치하기 위해서는 [GDAl,pyproj,Fiona,Shapely]와 같은 패키지를 먼저 설치 후 <br>
  geopandas 를 설치해야 geopandas 패키지를 사용할 수 있어 패키지의 Dependency 가 매우 높아 설치 시 많은 어려움이 따르고 있다. <br>
  
  이를 해결하기 위해 conda 에서 기본 패키지 다운로드 채널을 conda-forge 로 기본 채널 변경 후 <br>
**conda install geopandas** 로 설치하는 것을 적극 권장한다. (회사 내부 anaconda 유료 이슈도 참고하길 바란다)

  <img width="879" alt="스크린샷 2022-09-22 오후 5 33 21" src="https://user-images.githubusercontent.com/83544197/191699051-ceab12b4-461f-47c0-a755-3a25e5c07c38.png">

<br>

---

#### 1-2) import library <br>

In [2]:
import os 
import sys 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import geopandas as gpd 
import folium
from shapely.geometry import Point, MultiPoint, LineString, MultiLineString, Polygon, MultiPolygon



### **2. 파일 불러오기**
#### 2-1) shp 파일 불러오기 <br>

* 해당 [링크](http://www.gisdeveloper.co.kr/?p=2332) 에서 시도, 시군구, 행정동의 최신 shp 파일을 다운로드 
* geopandas 설치후 아래와 같이 데이터를 불러와 준다.
<br>


In [4]:
file_path = '/Applications/anaconda3/envs/gis_env/git/shape/data'
os.listdir(file_path)

['starbucks.csv',
 'seoul_starbucks.csv',
 'korea_sig_shp',
 '.DS_Store',
 'korea_adm_shp',
 'korea_sido_shp']

In [5]:
# 시도 
korea_sido_shp = gpd.read_file(
	file_path + '/korea_sido_shp/ctp_rvn.shp',
	encoding='EUC-KR'
)

# 시군구 
korea_sig_shp = gpd.read_file(
	file_path + '/korea_sig_shp/sig.shp',
	encoding='EUC-KR'
)

# 행정동 
korea_adm_shp = gpd.read_file(
	file_path + '/korea_adm_shp/emd.shp',
	encoding='EUC-KR'
)


---
#### 2-2) csv 파일 불러오기 <br>
* 해당 [링크](https://essim92.tistory.com/25) 에서 전국 스타벅스 가맹점 위치 데이터 다운로드 

In [25]:
korea_st_buck_data = pd.read_csv(
	file_path + '/starbucks.csv',
	encoding='EUC-KR'
)

korea_st_buck_data.head()

,지점명,주소,위도,경도
0,구로호텔,"서울특별시 구로구 디지털로32길 72, 구로호텔 1층 (구로동)",37.483129,126.898859
1,구로디지털타워,서울특별시 구로구 디지털로32길 30 (구로동),37.483432,126.896480
2,구로디지털로,"서울특별시 구로구 디지털로33길 11, 에이스테크노8차 1층 (구로동)",37.485911,126.895587
3,구로에이스,"서울특별시 구로구 디지털로 285, 에이스트윈타워1차 103호 (구로동)",37.484483,126.894496
4,구로하이엔드,"서울특별시 구로구 디지털로26길 5, 에이스하이엔드타워1차 103호 (구로동)",37.481618,126.893136


### **3. 공간 데이터 핸들링**
#### 3-1) 데이터 핸들링 <br>
#### 3-2) csv 파일의 x,y 컬럼을 공간 데이터(GeoDataFrame)으로 변환 <br>
#### 3-3) 좌표계 변환하기 <br>

<br>


#### 3-1) 데이터 핸들링 <br>


* 컬럼명 소문자로 변경 

In [18]:
# 컬럼명 소문자로 변경 

korea_sido_shp.columns = map(str.lower, korea_sido_shp.columns)
korea_sig_shp.columns = map(str.lower, korea_sig_shp.columns)
korea_adm_shp.columns = map(str.lower, korea_adm_shp.columns)

korea_sido_shp.head()

,ctprvn_cd,ctp_eng_nm,ctp_kor_nm,geometry
0,11,Seoul,서울특별시,"POLYGON ((966987.226 1941110.946, 966987.119 1..."
1,26,Busan,부산광역시,"MULTIPOLYGON (((1146778.390 1679623.999, 11469..."
2,27,Daegu,대구광역시,"POLYGON ((1087859.999 1760097.461, 1087859.951..."
3,28,Incheon,인천광역시,"MULTIPOLYGON (((897904.035 1961082.190, 897886..."
4,29,Gwangju,광주광역시,"POLYGON ((932712.687 1696168.692, 932781.680 1..."


* 한글 컬럼명 변겅 (개인 편의)

In [26]:

# 시도 
korea_sido_shp.rename(
	columns = {
		'ctprvn_cd': 'sido_cd',
		'ctp_eng_nm': 'sido_eng_nm',
		'ctp_kor_nm': 'sido_kor_nm'
	},
	inplace=True
)


#행정동 

korea_adm_shp.rename(
	columns={
		'emd_cd': 'adm_cd',
		'emd_eng_nm': 'adm_eng_nm',
		'emd_kor_nm': 'adm_kor_nm'
	},
	inplace=True
)


#스타벅스 

korea_st_buck_data.rename(
	columns = {
		'지점명': 'spot',
		'주소': 'address',
		'위도': 'lat',
		'경도': 'long'
	},
	inplace=True
)


---
#### 3-2) csv 파일의 x,y 컬럼을 공간 데이터(GeoDataFrame)으로 변환 <br>

* 위도(latitude), 경도(longitude) 각 컬럼을 아래와 같이 Point 형태의 데이터 타입으로 변경 (퍄트 5에서 추가 설명)

In [30]:
korea_st_buck_data['geometry'] = korea_st_buck_data.apply(
	lambda dt: Point([dt['long'],dt['lat']]),
	axis=1
)

korea_st_buck_data.head()

/Applications/anaconda3/envs/gis_env/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:128: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,spot,address,lat,long,geometry
0,구로호텔,"서울특별시 구로구 디지털로32길 72, 구로호텔 1층 (구로동)",37.483129,126.898859,POINT (126.89885909999998 37.4831293)
1,구로디지털타워,서울특별시 구로구 디지털로32길 30 (구로동),37.483432,126.896480,POINT (126.89648 37.483432)
2,구로디지털로,"서울특별시 구로구 디지털로33길 11, 에이스테크노8차 1층 (구로동)",37.485911,126.895587,POINT (126.895587 37.485911)
3,구로에이스,"서울특별시 구로구 디지털로 285, 에이스트윈타워1차 103호 (구로동)",37.484483,126.894496,POINT (126.8944957 37.48448329)
4,구로하이엔드,"서울특별시 구로구 디지털로26길 5, 에이스하이엔드타워1차 103호 (구로동)",37.481618,126.893136,POINT (126.893136 37.481618)


* GeoDataFrame 으로 변경시 'geometry' 파라미터에 공간 정보를 input 해줄 것 

In [41]:
korea_st_buck_gdf = gpd.GeoDataFrame(
	korea_st_buck_data,
	geometry = 'geometry'
)

korea_st_buck_gdf.drop(
	columns=['lat','long'],
	inplace=True
)

korea_st_buck_gdf.head()
type(korea_st_buck_gdf)

geopandas.geodataframe.GeoDataFrame

---
#### 3-3) 좌표계 부여, 변환하기 <br>

* 스타벅스 지점 데이터의 경우 초기 EPGS 값이 부여 되어 있지 않으므로 아래와 같이 EPGS 좌표계 부여

In [43]:
# 좌표계 부여
korea_st_buck_gdf.crs = {'init': 'epsg:4326'}

# 좌표계 확인
korea_st_buck_gdf.crs

/Applications/anaconda3/envs/gis_env/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


<Geographic 2D CRS: +init=epsg:4326 +type=crs>
Name: WGS 84
Axis Info [ellipsoidal]:
- lon[east]: Longitude (degree)
- lat[north]: Latitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

* 이제 스타벅스 지점 데이터와 shp 파일의 좌표계를 변경하여 같은 좌표계로 맞춰주자

In [ ]:
korea_sido_shp = korea_sido_shp.to_crs({'init': 'epsg:5179'})
korea_sig_shp = korea_sig_shp.to_crs({'init': 'epsg:5179'})
korea_adm_shp = korea_adm_shp.to_crs({'init': 'epsg:5179'})
korea_st_buck_gdf = korea_st_buck_gdf.to_crs({'init': 'epsg:5179'})

In [45]:
korea_sido_shp.head()

,sido_cd,sido_eng_nm,sido_kor_nm,geometry
0,11,Seoul,서울특별시,"POLYGON ((966987.226 1941110.946, 966987.119 1..."
1,26,Busan,부산광역시,"MULTIPOLYGON (((1146778.390 1679623.999, 11469..."
2,27,Daegu,대구광역시,"POLYGON ((1087859.999 1760097.461, 1087859.951..."
3,28,Incheon,인천광역시,"MULTIPOLYGON (((897904.035 1961082.190, 897886..."
4,29,Gwangju,광주광역시,"POLYGON ((932712.687 1696168.692, 932781.680 1..."


---
### **4. Geopandas로 공간 속성 다루기**


---
#### 4-1) 면적 (area) <br>


---
#### 4-2) 길이 <br>


---
#### 4-3) 도형의 무게중심 <br>


---
#### 4-4) boundary 확인 <br>


---
#### 4-5) 공간의 유효성 확인 <br>


---
#### 4-6) 유효한 도형으로 만들기 <br>


---
### **5. shapely 패키지 사용하여 shape 만들기**

---
#### 5-1) 점(Point)


---
#### 5-2) 선(LineString) 


---
#### 5-3) 도형(Polygon) 


---
#### 5-4) 한번에 여러 도형 만들기 (점,선,면))

---
### **6. 공간데이터 활용하기**

---
#### 6-1) 버퍼 (buffer)

---
#### 6-2) 도형을 감싸는 사각형 그리기(envelope)


---
#### 6-3) 도형 융해(dissolve)


---
#### 6-4) 도형끼리 겹치는 부분 처리하기(overlay)


---
### **7. 공간 데이터 결합**



#### 7-1) 여러 공간 결합 방법들
* within 


* contain

* intersects

* crosses

---
#### 7-2) 실제 데이터 응용하기


## **8. 저장하기**
#### 8-1) shp파일로 저장하기 <br>

<br>

---

## **9. Reference**
#### 9-1) ref <br>


[geopandas 사용법](https://yganalyst.github.io/spatial_analysis/spatial_analysis_2/)

[Shapely 사용법](https://programmerpsy.tistory.com/104)

[folinum 사용법](https://cow97.tistory.com/34)

[shp 파일 다운로드](http://www.gisdeveloper.co.kr/?p=2332)        
                                                                                                                                                                                                                                                                                       
<br>

---

* 다음은 낙동강 데이터를 이용한 실제 Geo data analysis 를 다뤄보도록 하겠다